In [1]:
import glob
import json
import re
import pickle
import pandas as pd
import spacy
import string
import numpy as np
import itertools

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process.kernels import RBF

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn.functional as F
import torchtext.vocab as vocab
nlp = spacy.load('en')

# Set the random seed manually for reproducibility.
torch.manual_seed(1234)

/people/maurice/anaconda3-5.1/envs/py3.6GPU/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/people/maurice/anaconda3-5.1/envs/py3.6GPU/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/people/maurice/anaconda3-5.1/envs/py3.6GPU/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/people/maurice/anaconda3-5.1/envs/py3.6GPU/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/people/maurice/anaconda3-5.1/envs/py3.6GPU/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary i

In [2]:
def save(nameFile, toSave):
    pickle_out = open(nameFile+".pickle", "wb")
    pickle.dump(toSave, pickle_out)
    pickle_out.close()

def load(nameFile):
    pickle_in = open(nameFile+".pickle", "rb")
    return pickle.load(pickle_in)

def main_iter_files():
    print('Import data')
    output_path = '/people/maurice/ownCloud/outputGentle/'
    wordsTimeds = []
    for file in sorted(glob.glob(output_path + '*')):
        print(file)
        if 'wordsTimed' in file and 'pickle' not in file:
            wordsTimed = pd.read_csv(file)  # load(file)
            print(wordsTimed.head())
            wordsTimeds.append(wordsTimed)
            # wordsTimedGby = wordsTimed.groupby('idSentence')
            '''sentenceTimed = wordsTimedGby.apply(lambda x: x.count())
            sentenceTimed[1] = sentenceTimed.astype(np.float)/len(g) 
            print sentenceTimed'''
    return wordsTimeds

In [3]:
wts = main_iter_files()

Import data
/people/maurice/ownCloud/outputGentle/align.pickle
/people/maurice/ownCloud/outputGentle/gentle_txt.pickle
/people/maurice/ownCloud/outputGentle/gentle_txtS2E23.pickle
/people/maurice/ownCloud/outputGentle/gentle_txtS2E5.pickle
/people/maurice/ownCloud/outputGentle/gntle_end.pickle
/people/maurice/ownCloud/outputGentle/gntle_endS2E23.pickle
/people/maurice/ownCloud/outputGentle/gntle_endS2E5.pickle
/people/maurice/ownCloud/outputGentle/gntle_start.pickle
/people/maurice/ownCloud/outputGentle/gntle_startS2E23.pickle
/people/maurice/ownCloud/outputGentle/gntle_startS2E5.pickle
/people/maurice/ownCloud/outputGentle/old
/people/maurice/ownCloud/outputGentle/phonesTimesS1E1.csv
/people/maurice/ownCloud/outputGentle/phonesTimesS1E10.csv
/people/maurice/ownCloud/outputGentle/phonesTimesS1E11.csv
/people/maurice/ownCloud/outputGentle/phonesTimesS1E12.csv
/people/maurice/ownCloud/outputGentle/phonesTimesS1E13.csv
/people/maurice/ownCloud/outputGentle/phonesTimesS1E14.csv
/people/mau

In [9]:
wts

[      Unnamed: 0        word  t_debut    t_fin  speaker  idSentence
 0              0          So     1.29     1.57  Sheldon           0
 1              1          if    -1.00    -1.00  Sheldon           0
 2              2           a     1.69     1.77  Sheldon           0
 3              3      photon     1.77     2.25  Sheldon           0
 4              4          is     2.26     2.37  Sheldon           0
 5              5    directed     2.37     2.72  Sheldon           0
 6              6     through     2.72     2.89  Sheldon           0
 7              7           a     2.89     2.94  Sheldon           0
 8              8       plane     2.94     3.29  Sheldon           0
 9              9        with     3.29     3.42  Sheldon           0
 10            10         two     3.44     3.60  Sheldon           0
 11            11       slits     3.60     3.91  Sheldon           0
 12            12          in     3.91     4.02  Sheldon           0
 13            13          it     

In [10]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [11]:
class sentenceTimed(object):
    def __init__(self, wt):
        self.reset(0)
        
    def reset(self, i):
        self.speaker = wt.iloc[i].speaker
        self.sentence_courante = ''
        if i > 0:
            self.sentence_courante += wt.iloc[i].word
        
    def modif_per_word(self, wt, i):
        self.sentence_courante += ' ' + wt.iloc[i].word

    def modif_per_sentence(self, wt, df, i):
        self.add_sentence_informations_to_dataframe(df)
        self.reset(i)
        
    def add_sentence_informations_to_dataframe(self, df):
        df.loc[len(df)] = [self.speaker, self.sentence_courante]
        # print(df)

In [7]:
# Lent
punctuation_end_sentence = ['!', '.', '?']
sentencesTimeds = []
print(len(wts))
for i, wt in enumerate(wts):
    print(i)
    sentencesTimed = pd.DataFrame(columns=['speaker', 'sentence_courante'])    

    st = sentenceTimed(wt)

    for word in range(len(wt)):
        if word == 0:
            st.modif_per_word(wt, word)
        elif wt.iloc[word].word[0].isupper() and wt.iloc[word - 1].word in punctuation_end_sentence:
            st.modif_per_sentence(wt, sentencesTimed, word)
        else:
            st.modif_per_word(wt, word)

    st.add_sentence_informations_to_dataframe(sentencesTimed)
    sentencesTimeds.append(sentencesTimed)

39
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38


In [12]:
sentencesTimeds[0]

,speaker,sentence_courante
0,Sheldon,So if a photon is directed through a plane wi...
1,Sheldon,"If it 's unobserved it will , however , if it ..."
2,Leonard,"Agreed , what 's your point ?"
3,Sheldon,"There 's no point , I just think it 's a good ..."
4,Leonard,Excuse me ?
5,Receptionist,Hang on .
6,Leonard,"One across is Aegean , eight down is Nabakov ,..."
7,Leonard,"See , Papa Doc 's capital idea , that 's Port ..."
8,Leonard,Haiti .
9,Receptionist,Can I help you ?


In [13]:
sentencesTimeds[0].iloc[1]

speaker                                                        Sheldon
sentence_courante    If it 's unobserved it will , however , if it ...
Name: 1, dtype: object

In [14]:
sentencesTimeds[0].iloc[1].speaker

'Sheldon'

In [15]:
sentencesTimeds[0].iloc[1].sentence_courante

"If it 's unobserved it will , however , if it 's observed after it 's left the plane but before it hits its target , it will not have gone through both slits ."

In [16]:
data = pd.concat(sentencesTimeds)

In [17]:
data

,speaker,sentence_courante
0,Sheldon,So if a photon is directed through a plane wi...
1,Sheldon,"If it 's unobserved it will , however , if it ..."
2,Leonard,"Agreed , what 's your point ?"
3,Sheldon,"There 's no point , I just think it 's a good ..."
4,Leonard,Excuse me ?
5,Receptionist,Hang on .
6,Leonard,"One across is Aegean , eight down is Nabakov ,..."
7,Leonard,"See , Papa Doc 's capital idea , that 's Port ..."
8,Leonard,Haiti .
9,Receptionist,Can I help you ?


In [18]:
data.speaker.values

array(['Sheldon', 'Sheldon', 'Leonard', ..., 'Computer voice', 'Penny',
       'Computer voice'], dtype=object)

In [19]:
data.sentence_courante.values

array([' So if a photon is directed through a plane with two slits in it and either slit is observed it will not go through both slits .',
       "If it 's unobserved it will , however , if it 's observed after it 's left the plane but before it hits its target , it will not have gone through both slits .",
       "Agreed , what 's your point ?", ...,
       'Some hiney would be nice , too .', 'Hiney ?', 'Honey .'],
      dtype=object)

In [20]:
X = [s.lower().split() for s in data.sentence_courante.values]
Y = [s.lower() for s in data.speaker.values]
newY = np.zeros(len(Y)-1)
for i in range(1, len(Y)):
    if Y[i] == Y[i-1]:
        newY[i-1] = 0
    else:
        newY[i-1] = 1
Y = Variable(torch.from_numpy(newY))

In [21]:
X = [s.lower().split() for s in data.sentence_courante.values]
Y = [s.lower() for s in data.speaker.values]
newY = []
for i in range(1, len(Y)):
    if Y[i] == Y[i-1]:
        newY.append([0])
    else:
        newY.append([1])
Y = Variable(torch.FloatTensor(newY))

In [22]:
print(X[0:2])
print(Y)
print(Y.shape)

[['so', 'if', 'a', 'photon', 'is', 'directed', 'through', 'a', 'plane', 'with', 'two', 'slits', 'in', 'it', 'and', 'either', 'slit', 'is', 'observed', 'it', 'will', 'not', 'go', 'through', 'both', 'slits', '.'], ['if', 'it', "'s", 'unobserved', 'it', 'will', ',', 'however', ',', 'if', 'it', "'s", 'observed', 'after', 'it', "'s", 'left', 'the', 'plane', 'but', 'before', 'it', 'hits', 'its', 'target', ',', 'it', 'will', 'not', 'have', 'gone', 'through', 'both', 'slits', '.']]
tensor([[0.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]])
torch.Size([12701, 1])


In [23]:
#embed = nn.Embedding(num_embeddings, embedding_dim)
# pretrained_weight is a numpy matrix of shape (num_embeddings, embedding_dim)
#embed.weight.data.copy_(torch.from_numpy(pretrained_weight))

#we = vocab.GloVe(name='6B', dim=100)
we = vocab.FastText(language='en')
'''
pretrained_aliases = {
    "charngram.100d": partial(CharNGram),
    "fasttext.en.300d": partial(FastText, language="en"),
    "fasttext.simple.300d": partial(FastText, language="simple"),
    "glove.42B.300d": partial(GloVe, name="42B", dim="300"),
    "glove.840B.300d": partial(GloVe, name="840B", dim="300"),
    "glove.twitter.27B.25d": partial(GloVe, name="twitter.27B", dim="25"),
    "glove.twitter.27B.50d": partial(GloVe, name="twitter.27B", dim="50"),
    "glove.twitter.27B.100d": partial(GloVe, name="twitter.27B", dim="100"),
    "glove.twitter.27B.200d": partial(GloVe, name="twitter.27B", dim="200"),
    "glove.6B.50d": partial(GloVe, name="6B", dim="50"),
    "glove.6B.100d": partial(GloVe, name="6B", dim="100"),
    "glove.6B.200d": partial(GloVe, name="6B", dim="200"),
    "glove.6B.300d": partial(GloVe, name="6B", dim="300")
}
'''

def get_word_vector(word):
    return we.vectors[we.stoi[word]]

def closest(vec, n=2):#10):
    """
    Find the closest words for a given vector
    """
    all_dists = [(w, torch.dist(vec, get_word_vector(w))) for w in we.itos]
    return sorted(all_dists, key=lambda t: t[1])[:n]

def print_tuples(tuples):
    for tuple in tuples:
        print('(%.4f) %s' % (tuple[1], tuple[0]))

# In the form w1 : w2 :: w3 : ?
def analogy(w1, w2, w3, n=5, filter_given=True):
    print('\n[%s : %s :: %s : ?]' % (w1, w2, w3))
   
    # w2 - w1 + w3 = w4
    closest_words = closest(get_word_vector(w2) - get_word_vector(w1) + get_word_vector(w3))
    
    # Optionally filter out given words
    if filter_given:
        closest_words = [t for t in closest_words if t[0] not in [w1, w2, w3]]

    print_tuples(closest_words[:n])

In [ ]:
# Lent
print(we.dim,'\n')
print_tuples(closest(get_word_vector('google')))
analogy('king', 'man', 'queen')
print(type(we.stoi),'\n')
a = 0
for k, v in we.stoi.items():
    if a < 10:
        print(k,v)
        a += 1
    else:
        break
print('iazkcnzejjqsdchj' in we.stoi)

300 



In [24]:
nunf = set()
for s in X:
    for w in s:#.split():
        if w not in we.stoi:
            nunf.add(w)
print(len(nunf), nunf)

392 {'yousa', 'i.e', 'punchali', 'humormometer', 'elpful', 'n=4', 'c’mon', 'x10', '18th', 'vh1', '11', 'bilurrrbe', '2099', 'friendmaking', 'boys’', '130', '1908', 'afwaid', '45', 'woboto', 'cwazy', 'slurpies', 'pruny', 'medpack', '800', '320', 'x10s', 'weducing', '51', 'wubber', '1992', '21st', '1980', 'anobo', 'charmalarmalon', '1974', 'sezchuan', 'flankenzie', 's’u’up', 'pumiced', 'dewusions', 'd’être', 'y.', 'schmort', 'appreeee', 'neeeeooooowwwww', '1778', 'quantii', '1487', '1,000', 'darmspülung', 'spewers', '170', 'licquorish', 'kwipke', 'finkleday', 'fertiliziation', 'contwol', 'sheldor', '29', '2008', "'em", '4', '140', '60', 'mr.', 'mytosis', 'strippergram', 'wesults', 'neeeeeooooowwwwww', '24', 'd.', 'mcfloonyloo', 'impwessive', 'swrap', 'allergenics', '’ll', 'kwippler', 'ekairidium', '75', 'wolowizard', 'cherrrkh', 'thwee', 'webchatting', 'whatchacall', '12.50', 'fuzzyboots', 'bawy', '12th', '360', 'under-30', '99', '17', '187', 'q’wass', 'sluttiness', '2328', '’cause', '19

In [25]:
'''weX = []
for i,s in enumerate(X):
    weX.append([])
    for w in s.split():
        if w in we.stoi:
            weX[i].append(get_word_vector(w))'''

to_del = []
for s in X:
    for w in s:
        if w not in we.stoi:
            to_del.append(w)
X = [[w for w in s if w not in to_del] for s in X]

In [26]:
print(X[0:2])

[['so', 'if', 'a', 'photon', 'is', 'directed', 'through', 'a', 'plane', 'with', 'two', 'slits', 'in', 'it', 'and', 'either', 'slit', 'is', 'observed', 'it', 'will', 'not', 'go', 'through', 'both', 'slits', '.'], ['if', 'it', 'unobserved', 'it', 'will', ',', 'however', ',', 'if', 'it', 'observed', 'after', 'it', 'left', 'the', 'plane', 'but', 'before', 'it', 'hits', 'its', 'target', ',', 'it', 'will', 'not', 'have', 'gone', 'through', 'both', 'slits', '.']]


In [27]:
words_set = set()
for s in X:
    words_set = words_set.union(set(s))
print(len(words_set))

8842


In [28]:
we_idx = [we.stoi[w] for w in list(words_set)]
we_idx.append(0) #for padding we need to intialize one row of vector weights

In [29]:
# map sentences to vocab
idw = 1
vocab_X = {'<PAD>':0}
for w in list(words_set):
    vocab_X[w] = idw
    idw += 1

# fancy nested list comprehension
X_num =  [[vocab_X[word] for word in sentence] for sentence in X]

print(len(vocab_X))
print(X[0:2])

8843
[['so', 'if', 'a', 'photon', 'is', 'directed', 'through', 'a', 'plane', 'with', 'two', 'slits', 'in', 'it', 'and', 'either', 'slit', 'is', 'observed', 'it', 'will', 'not', 'go', 'through', 'both', 'slits', '.'], ['if', 'it', 'unobserved', 'it', 'will', ',', 'however', ',', 'if', 'it', 'observed', 'after', 'it', 'left', 'the', 'plane', 'but', 'before', 'it', 'hits', 'its', 'target', ',', 'it', 'will', 'not', 'have', 'gone', 'through', 'both', 'slits', '.']]


In [30]:
# get the length of each sentence
X_lengths = [len(sentence) for sentence in X_num]
print(X_lengths[0:2])

# create an empty matrix with padding tokens
padding_idx = vocab_X['<PAD>']
longest_sent = max(X_lengths)
print(longest_sent)
batch_size = len(X_num)
padded_X = np.ones((batch_size, longest_sent)) * padding_idx

# copy over the actual sequences
for i, x_len in enumerate(X_lengths):
    sequence = X_num[i]
    padded_X[i, 0:x_len] = sequence[:x_len]

# padded_X looks like:
print(padded_X[0:2][:])

[27, 32]
109
[[2869.  782. 7686. 2126.  591. 6245. 5823. 7686. 1026. 4200. 8377.  691.
  5450. 3236. 7742. 1339. 3146.  591. 4996. 3236.  898. 6747. 3094. 5823.
  7503.  691. 2943.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.]
 [ 782. 3236. 2137. 3236.  898. 7056. 6933. 7056.  782. 3236. 4996. 4470.
  3236. 1319. 5097. 1026. 8468. 5910. 3236. 7011. 7127. 5986. 7056. 3236.
   898. 6747. 8757. 5292. 5823. 7503.  691. 2943.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.   

In [26]:
inp = Variable(torch.LongTensor([[3,4,6], [1,3,5]]))
#inp = Variable(torch.LongTensor([3,4]))
print(inp)
embed(inp)

tensor([[3, 4, 6],
        [1, 3, 5]])


NameError: name 'embed' is not defined

In [27]:
list(range(5,2,-1))

[5, 4, 3]

In [55]:
#https://gist.github.com/Tushar-N/dfca335e370a2bc3bc79876e6270099e

def get_last_modulo(nb, mod):
    if nb%mod == 0:
        return nb
    else:
        return int(nb/mod) * mod

### Premier modèle, somme des embedding par phrases
taille_embedding = len(get_word_vector(X[0][0]))
taille_context = 3

idx_set_words = dict(zip(list(words_set), range(1,len(words_set)+1)))
idx_set_words['<PAD>'] = 0
padding_idx = idx_set_words['<PAD>']
vectorized_seqs = [[idx_set_words[w] for w in s]for s in X]
#vectorized_seqs = padded_X
print(vectorized_seqs[0:2])
embed = nn.Embedding(num_embeddings=len(words_set)+1, embedding_dim=taille_embedding, padding_idx=padding_idx)
embed.weight.data.copy_(we.vectors[we_idx])
embed.weight.requires_grad = False

# get the length of each seq in your batch
seq_lengths = torch.LongTensor(list(map(len, vectorized_seqs)))
print('length', seq_lengths)
# dump padding everywhere, and place seqs on the left.
# NOTE: you only need a tensor as big as your longest sequence
seq_tensor = Variable(torch.zeros((len(vectorized_seqs), seq_lengths.max()))).long()
for idx, (seq, seqlen) in enumerate(zip(vectorized_seqs, seq_lengths)):
    seq_tensor[idx, :seqlen] = torch.LongTensor(seq)

print(seq_tensor[0:2])
print('nb sentences', len(vectorized_seqs))

# utils.rnn lets you give (B,L,D) tensors where B is the batch size, L is the maxlength, if you use batch_first=True
# Otherwise, give (L,B,D) tensors
seq_tensor = seq_tensor.transpose(0,1) # (B,L,D) -> (L,B,D)

# embed your sequences
seq_tensor = embed(seq_tensor)

# sum over L, all words per sentence
seq_tensor_sumed = torch.sum(seq_tensor, dim=0) #len(vectorized_seqs), taille_embedding
print('after sum', seq_tensor_sumed.shape)
seq_tensor_sumed = seq_tensor_sumed.view(len(vectorized_seqs), 1, taille_embedding)
#seq_tensor_sumed = seq_tensor_sumed.view(seq_tensor_sumed.shape[0], 1, taille_embedding)
#seq_tensor_sumed = seq_tensor_sumed.view(int(get_last_modulo(seq_tensor_sumed.shape[0], taille_context)/taille_context), taille_context, seq_tensor_sumed.shape[1])
print(seq_tensor_sumed.shape)
print('sum', seq_tensor_sumed)

bidirectional = False
input_size = taille_embedding
hidden_size = taille_embedding
num_layers = 3
nb_sentences = len(vectorized_seqs)
lstm_previous = torch.nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=False, bidirectional=bidirectional)
lstm_future = torch.nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=False, bidirectional=bidirectional)
tanh = nn.Tanh()
softmax = nn.Softmax()
#get mini-batch
for i in range(nb_sentences - (2*taille_context - 1) -1): #TODO split train/test/val
    indices_previous = torch.tensor(list(range(i,i+taille_context+1)))
    indices_future = torch.tensor(list(range(i+2*taille_context+1,i+taille_context,-1)))
    input_previous_features = torch.index_select(seq_tensor_sumed, 0, indices_previous)
    input_future_features = torch.index_select(seq_tensor_sumed, 0, indices_future)
    seq_tensor_output_previous, (ht_previous, ct_previous) = lstm_previous(input_previous_features)
    seq_tensor_output_future, (ht_future, ct_future) = lstm_future(input_future_features)
    target = Y[i + taille_context]
    seq_len = input_previous_features.shape[0]
    batch = input_previous_features.shape[1]
    #print(input_previous_features.shape) # 4 1 300
    num_directions = 1
    if bidirectional:
        num_directions = 2
    seq_tensor_output_sum = torch.cat((seq_tensor_output_previous.view(seq_len, batch, num_directions, hidden_size)[-1], seq_tensor_output_future.view(seq_len, batch, num_directions, hidden_size)[-1]), -1)
    print(seq_tensor_output_sum[-1].shape, seq_tensor_output_previous.shape, seq_tensor_output_previous.view(seq_len, batch, num_directions, hidden_size).shape, seq_tensor_output_previous[-1].shape, seq_tensor_output_future[-1].shape, seq_tensor_output_sum.shape, target)
    #torch.Size([1, 600]) torch.Size([4, 1, 300]) torch.Size([4, 1, 1, 300]) torch.Size([1, 300]) torch.Size([1, 300]) torch.Size([1, 1, 600]) tensor(1., dtype=torch.float64)
    #TODO Attention mechanism
    seq_tensor_output_sum = seq_tensor_output_sum.view(1,2*num_directions*taille_embedding) #2 is because we concatenate previous and future embeddings
    print(seq_tensor_output_sum.shape)
    W = torch.rand(1, num_directions)
    print(W.shape, seq_tensor_output_sum.shape, W.mm(seq_tensor_output_sum).shape)
    print(tanh(W.mm(seq_tensor_output_sum)).shape)
    u = torch.rand(1, 2*taille_embedding)
    print(softmax(u.mm(tanh(W.mm(seq_tensor_output_sum)).t())))
    break
# throw them through your LSTM (remember to give batch_first=True here if you packed with it)
#lstm = torch.nn.LSTM(input_size=seq_tensor_sumed.shape[2], hidden_size=seq_tensor_sumed.shape[2], num_layers=1, batch_first=False, bidirectional=False)
#print(lstm)
#seq_tensor_output, (ht, ct) = lstm(seq_tensor_sumed)
#print('output', seq_tensor_output)

# Or if you just want the final hidden state?
#print(ht[-1])

[[4169, 7342, 5962, 8437, 5164, 7482, 1566, 5962, 2918, 5311, 4970, 2797, 6516, 7058, 2511, 3073, 323, 5164, 8175, 7058, 870, 3717, 6986, 1566, 6629, 2797, 5992], [7342, 7058, 1437, 7058, 870, 6062, 4562, 6062, 7342, 7058, 8175, 7461, 7058, 8475, 129, 2918, 6632, 6473, 7058, 5287, 1649, 2816, 6062, 7058, 870, 3717, 5766, 8358, 1566, 6629, 2797, 5992]]
lenght tensor([27, 32,  6,  ...,  8,  2,  2])
tensor([[4169, 7342, 5962, 8437, 5164, 7482, 1566, 5962, 2918, 5311, 4970, 2797,
         6516, 7058, 2511, 3073,  323, 5164, 8175, 7058,  870, 3717, 6986, 1566,
         6629, 2797, 5992,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0, 

/people/maurice/anaconda3-5.1/envs/py3.6GPU/lib/python3.6/site-packages/ipykernel_launcher.py:84: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [31]:
class HierarchicalBiLSTM_on_sentence_embedding(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, targset_size, num_layers = 3, bidirectional = False):
        super(HierarchicalBiLSTM_on_sentence_embedding, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.bidirectional = bidirectional
        self.num_directions = 1
        if self.bidirectional:
            self.num_directions = 2
        self.num_layers = num_layers

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm_previous = torch.nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=num_layers, batch_first=False, bidirectional=bidirectional)
        self.lstm_future = torch.nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=num_layers, batch_first=False, bidirectional=bidirectional)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(2*self.num_directions*hidden_dim, targset_size)#2 because we concatenate the both output of the lstm
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(self.num_layers, 1, self.hidden_dim),
                torch.zeros(self.num_layers, 1, self.hidden_dim))

    def forward(self, input_previous_sentences, input_future_sentences):
        seq_tensor_output_previous, self.hidden = self.lstm_previous(input_previous_sentences, self.hidden)
        seq_tensor_output_future, self.hidden = self.lstm_future(input_future_sentences, self.hidden)
        
        seq_len = input_previous_sentences.shape[0]
        batch = input_previous_sentences.shape[1]

        seq_tensor_output_sum = torch.cat((seq_tensor_output_previous.view(seq_len, batch, self.num_directions, self.hidden_dim)[-1,:,:], seq_tensor_output_future.view(seq_len, batch, self.num_directions, self.hidden_dim)[-1,:,:]), -1)
        #TODO Attention mechanism
        seq_tensor_output_sum = seq_tensor_output_sum.view(batch,2*self.num_directions*self.hidden_dim) #2 is because we concatenate previous and future embeddings

        #lstm_out, self.hidden = self.lstm(embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(seq_tensor_output_sum)
        tag_space = tag_space[0]
        prediction = torch.sigmoid(tag_space)
        return prediction

In [ ]:
#https://gist.github.com/Tushar-N/dfca335e370a2bc3bc79876e6270099e

import torch.optim as optim

### Premier modèle, somme des embedding par phrases
taille_embedding = len(get_word_vector(X[0][0]))
taille_context = 3
bidirectional = False
num_layers = 3
nb_epoch = 5
targset_size = 1

idx_set_words = dict(zip(list(words_set), range(1,len(words_set)+1)))
idx_set_words['<PAD>'] = 0
padding_idx = idx_set_words['<PAD>']
vectorized_seqs = [[idx_set_words[w] for w in s]for s in X]
#vectorized_seqs = padded_X
print(vectorized_seqs[0:2])
embed = nn.Embedding(num_embeddings=len(words_set)+1, embedding_dim=taille_embedding, padding_idx=padding_idx)
embed.weight.data.copy_(we.vectors[we_idx])
embed.weight.requires_grad = False

# get the length of each seq in your batch
seq_lengths = torch.LongTensor(list(map(len, vectorized_seqs)))
print('length', seq_lengths)
# dump padding everywhere, and place seqs on the left.
# NOTE: you only need a tensor as big as your longest sequence
seq_tensor = Variable(torch.zeros((len(vectorized_seqs), seq_lengths.max()))).long()
for idx, (seq, seqlen) in enumerate(zip(vectorized_seqs, seq_lengths)):
    seq_tensor[idx, :seqlen] = torch.LongTensor(seq)

print(seq_tensor[0:2])
print('nb sentences', len(vectorized_seqs))

# utils.rnn lets you give (B,L,D) tensors where B is the batch size, L is the maxlength, if you use batch_first=True
# Otherwise, give (L,B,D) tensors
seq_tensor = seq_tensor.transpose(0,1) # (B,L,D) -> (L,B,D)

# embed your sequences
seq_tensor = embed(seq_tensor)

# sum over L, all words per sentence
seq_tensor_sumed = torch.sum(seq_tensor, dim=0) #len(vectorized_seqs), taille_embedding
print('after sum', seq_tensor_sumed.shape)
seq_tensor_sumed = seq_tensor_sumed.view(len(vectorized_seqs), 1, taille_embedding)
#seq_tensor_sumed = seq_tensor_sumed.view(seq_tensor_sumed.shape[0], 1, taille_embedding)
#seq_tensor_sumed = seq_tensor_sumed.view(int(get_last_modulo(seq_tensor_sumed.shape[0], taille_context)/taille_context), taille_context, seq_tensor_sumed.shape[1])
print(seq_tensor_sumed.shape)
print('sum', seq_tensor_sumed)

bidirectional = False
input_size = taille_embedding
hidden_size = taille_embedding
num_layers = 3
nb_sentences = len(vectorized_seqs)

model = HierarchicalBiLSTM_on_sentence_embedding(taille_embedding, taille_embedding, targset_size, num_layers, bidirectional)
loss_function = nn.BCELoss()#NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

print('début train')
for epoch in range(nb_epoch):
    #get mini-batch
    #Data loader
    for i in range(nb_sentences - (2*taille_context - 1) -1): #TODO split train/test/val
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Also, we need to clear out the hidden state of the LSTM,
        # detaching it from its history on the last instance.
        model.hidden = model.init_hidden()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        indices_previous = torch.tensor(list(range(i,i+taille_context+1)))
        indices_future = torch.tensor(list(range(i+2*taille_context+1,i+taille_context,-1)))
        input_previous_features = torch.index_select(seq_tensor_sumed, 0, indices_previous)
        input_future_features = torch.index_select(seq_tensor_sumed, 0, indices_future)
        
        # Step 3. Run our forward pass.
        prediction = model(input_previous_features, input_future_features)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(prediction, Y[i+taille_context]) #targets)
        loss.backward()
        optimizer.step()
        
        #break
    #break
print('fin train')
# See what the scores are after training
with torch.no_grad():
    i = 0
    indices_previous = torch.tensor(list(range(i,i+taille_context+1)))
    indices_future = torch.tensor(list(range(i+2*taille_context+1,i+taille_context,-1)))
    input_previous_features = torch.index_select(seq_tensor_sumed, 0, indices_previous)
    input_future_features = torch.index_select(seq_tensor_sumed, 0, indices_future)
    prediction = model(input_previous_features, input_future_features)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(prediction, Y[i+taille_context])

[[2869, 782, 7686, 2126, 591, 6245, 5823, 7686, 1026, 4200, 8377, 691, 5450, 3236, 7742, 1339, 3146, 591, 4996, 3236, 898, 6747, 3094, 5823, 7503, 691, 2943], [782, 3236, 2137, 3236, 898, 7056, 6933, 7056, 782, 3236, 4996, 4470, 3236, 1319, 5097, 1026, 8468, 5910, 3236, 7011, 7127, 5986, 7056, 3236, 898, 6747, 8757, 5292, 5823, 7503, 691, 2943]]
length tensor([27, 32,  6,  ...,  8,  2,  2])
tensor([[2869,  782, 7686, 2126,  591, 6245, 5823, 7686, 1026, 4200, 8377,  691,
         5450, 3236, 7742, 1339, 3146,  591, 4996, 3236,  898, 6747, 3094, 5823,
         7503,  691, 2943,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0, 

In [34]:
with torch.no_grad():
    i = 0
    indices_previous = torch.tensor(list(range(i,i+taille_context+1)))
    indices_future = torch.tensor(list(range(i+2*taille_context+1,i+taille_context,-1)))
    input_previous_features = torch.index_select(seq_tensor_sumed, 0, indices_previous)
    input_future_features = torch.index_select(seq_tensor_sumed, 0, indices_future)
    prediction = model(input_previous_features, input_future_features)
    print(prediction, Y[i+taille_context])

tensor([0.6674]) tensor([1.])


In [28]:
print(torch.__version__)
embed = nn.Embedding(len(words_set), len(get_word_vector(X[0][0])))
embed.weight.data.copy_(we.vectors[we_idx])
embed.weight.requires_grad = False

idx_set_words = dict(zip(list(words_set), range(len(words_set))))

vectorized_seqs = [[idx_set_words[w] for w in s]for s in X]
print(vectorized_seqs[0:2])

# get the length of each seq in your batch
seq_lengths = torch.LongTensor(list(map(len, vectorized_seqs)))
print(seq_lengths)
# dump padding everywhere, and place seqs on the left.
# NOTE: you only need a tensor as big as your longest sequence
seq_tensor = Variable(torch.zeros((len(vectorized_seqs), seq_lengths.max()))).long()
for idx, (seq, seqlen) in enumerate(zip(vectorized_seqs, seq_lengths)):
    seq_tensor[idx, :seqlen] = torch.LongTensor(seq)

# SORT YOUR TENSORS BY LENGTH!
seq_lengths, perm_idx = seq_lengths.sort(0, descending=True)
seq_tensor = seq_tensor[perm_idx]

# utils.rnn lets you give (B,L,D) tensors where B is the batch size, L is the maxlength, if you use batch_first=True
# Otherwise, give (L,B,D) tensors
seq_tensor = seq_tensor.transpose(0,1) # (B,L,D) -> (L,B,D)
print(seq_tensor)

# embed your sequences
seq_tensor = embed(seq_tensor)
print(seq_tensor)

# pack them up nicely
packed_input = pack_padded_sequence(seq_tensor, seq_lengths.cpu().numpy())
print(packed_input)

# throw them through your LSTM (remember to give batch_first=True here if you packed with it)
packed_output, (ht, ct) = lstm(packed_input)
print(packed_output)

# unpack your output if required
output, _ = pad_packed_sequence(packed_output)
print(output)

# Or if you just want the final hidden state?
print(ht[-1])

0.4.0
[[6597, 3994, 6329, 2163, 7702, 5246, 8520, 6329, 149, 3939, 818, 4668, 6880, 1140, 8191, 4768, 4629, 7702, 5182, 1140, 3685, 8087, 837, 8520, 1533, 4668, 8367], [3994, 1140, 7151, 1140, 3685, 3336, 7496, 3336, 3994, 1140, 5182, 6006, 1140, 6507, 934, 149, 773, 8034, 1140, 1426, 3684, 3737, 3336, 1140, 3685, 8087, 6252, 702, 8520, 1533, 4668, 8367]]
tensor([  27,   32,    6,  ...,    8,    2,    2])
tensor([[  934,  2177,  2526,  ...,  1903,  8367,  8367],
        [ 6089,  2812,  3336,  ...,     0,     0,     0],
        [ 5781,  5567,  2177,  ...,     0,     0,     0],
        ...,
        [ 5853,     0,     0,  ...,     0,     0,     0],
        [ 2167,     0,     0,  ...,     0,     0,     0],
        [ 6124,     0,     0,  ...,     0,     0,     0]])
tensor([[[-6.5334e-02, -9.3031e-02, -1.7571e-02,  ...,  1.6642e-01,
          -1.3079e-01,  3.5397e-02],
         [-4.0083e-01,  1.3992e-01,  1.2907e-01,  ...,  2.3206e-01,
           7.9414e-02,  1.3813e-01],
         [-3.6183e-

NameError: name 'lstm' is not defined

In [29]:
def flatten(l):
    return list(itertools.chain.from_iterable(l))

seqs = ['ghatmasala','nicela','c-pakodas']

# make <pad> idx 0
vocab = ['<pad>'] + sorted(list(set(flatten(seqs))))
print(vocab)

vectorized_seqs = [[vocab.index(tok) for tok in seq]for seq in seqs]

# get the length of each seq in your batch
seq_lengths = map(len, vectorized_seqs)
print(list(seq_lengths))

['<pad>', '-', 'a', 'c', 'd', 'e', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 's', 't']
[10, 6, 9]


In [30]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1):
        super(EncoderRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        
    def forward(self, word_inputs, hidden):
        # Note: we run this all at once (over the whole input sequence)
        seq_len = len(word_inputs)
        embedded = self.embedding(word_inputs).view(seq_len, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def init_hidden(self):
        hidden = Variable(torch.zeros(self.n_layers, 1, self.hidden_size))
        if USE_CUDA: hidden = hidden.cuda()
        return hidden

In [ ]:
#https://www.kdnuggets.com/2018/06/taming-lstms-variable-sized-mini-batches-pytorch.html
#https://github.com/EdGENetworks/attention-networks-for-classification/blob/master/model.py
#https://github.com/EdGENetworks/attention-networks-for-classification/blob/master/attention_model_validation_experiments.ipynb
#https://explosion.ai/blog/deep-learning-formula-nlp
#https://github.com/koustuvsinha/hred-py/blob/master/hred_pytorch.py